In [ ]:
import asyncio
import aiohttp
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np
import re
import string
from IPython.display import clear_output

def list_spliter_by_num(target_list, number):
    """
    넣은 리스트를 number 단위로 쪼개는 함수
    """
    for i in range(0, len(target_list), number):
        yield target_list[i:i+number]

async def get_text_from_url(url):
    async with aiohttp.ClientSession() as session: # 다 끝나면 세션을 닫을것이다.
        async with session.get(url, headers={'user-agent': 'Mozilla/5.0'}) as req: # 다 끝나면 get 요청도 닫을것이다.
            html = await req.text() # 여기서 시간이 오래 걸리므로 기다린다.
    
    soup = BeautifulSoup(html, 'lxml')
    
    try :
        main_txt = soup.select_one('div#articletxt').get_text()
    except Exception as e :
        main_txt = np.nan
    
    return url, main_txt
    
async def main():
    # 저장할 파일 헤더 리스트
    header_list = ['기사 URL', '기사 내용']
    # 저장할 경로
    filepath = './data/'
    journal = '한국경제'
    
    try :
        news_df = pd.read_csv(filepath + 'hk_news_url_part1.csv', encoding = 'utf-8', dtype = {'종목코드' : object})
        try :
            main_df = pd.read_csv(filepath + 'hk_news_main.csv', encoding = 'utf-8', dtype = {'종목코드' : object})
        except Exception as e :
            main_df = pd.DataFrame(columns= header_list)
            main_df.to_csv(filepath + 'hk_news_main.csv', encoding = 'utf-8', index = False)
        
        # 기사 url 중복 제거
        copy_df = news_df.drop_duplicates(['기사 URL'])
        copy_df.set_index('기사 URL', inplace = True)
        search_df = copy_df.loc[set(news_df['기사 URL']) - set(main_df['기사 URL'])]
        search_df.reset_index(inplace = True)
        
        url_list = search_df['기사 URL'].dropna().tolist()
        split_list = list_spliter_by_num(url_list, 10)
        
        print('남은 기사수 : ', len(url_list))
        
        # 진행상황 체크
        progress = 0
        
        while True :
            mk_list = [] # 크롤링된 데이터 리스트
            error_list = [] # 에러난 기업 데이터 리스트
            try :
                try :
                    tasks = []
                    e_url = ''
                    try : 
                        urls = next(split_list)
                        for url in urls :
                            tasks.append(get_text_from_url(url))
                        result = await asyncio.gather(*tasks)
                        # 리스트에 담아주기
                        for idx in range(0, 10) :
                            mk_list.append([result[idx][0], result[idx][1]]) # 0 : url, 1 : 본문 내용
                            
                    except StopIteration as s :
                        return 1
                    except Exception as e :
                        print('whlie문 에러 : ', e)
                        print('=' * 60)
                        # 파일 저장
                    mt_df = pd.DataFrame(mk_list, columns = ['기사 URL', '기사 내용'])
                    mt_df.to_csv(filepath + 'hk_news_main.csv', mode = 'a', index = False, encoding = 'utf-8', header = False)
                    progress += 10
                    print('{} 기사 수집완료'.format(progress))
                except StopIteration :
                    print('모든 URL에 대한 수집이 완료되었습니다.')
                    return 1
            except Exception as e :
                print('에러 : ', e)
                return 0
                
    except Exception as e : 
        print('파일읽기 실패 : ', e)

In [ ]:
#####################################################
############### 기사 내용 수집부분 ###################
#####################################################
while True :
    clear_output(wait = True)
    start = time.time()
    result = await main()
    end = time.time()
    
    print(f'걸린 시간 : {end-start}')
    if result == 1 :
        break
    else :
        continue

# 기사 수 확인 비교

In [1]:
import pandas as pd

# 저장할 파일 헤더 리스트
header_list = ['기사 URL', '기사 내용']
# 저장할 경로
filepath = './data/'

news_df = pd.read_csv(filepath + 'hk_news_url.csv', encoding = 'utf-8', dtype = {'종목코드' : object})
try :
    main_df = pd.read_csv(filepath + 'hk_news_main.csv', encoding = 'utf-8', dtype = {'종목코드' : object})
except Exception as e :
    main_df = pd.DataFrame(columns= header_list)
    main_df.to_csv(filepath + 'hk_news_main.csv', encoding = 'utf-8', index = False)

# 기사 url 중복 제거
copy_df = news_df.drop_duplicates(['기사 URL'])
copy_df.set_index('기사 URL', inplace = True)
#         search_df = copy_df.loc[set(news_df['기사 URL']) - set(main_df['기사 URL'])]
search_df = copy_df.loc[set(news_df['기사 URL']) - set(main_df['기사 URL'])]
search_df.reset_index(inplace = True)

In [2]:
len(news_df['종목코드'].unique())

2709

In [3]:
len(news_df['기사 URL'].unique())

209947

# 기사 url 데이터, 기사 본문 데이터 merge

In [5]:
result = pd.merge(news_df, main_df)

In [7]:
result.dropna(inplace = True)

In [8]:
result

,기업명,기사 제목,기사 URL,언론사명,기사 업로드 날짜,종목코드,기사 내용
0,굿모닝신한증권,강세場선 '달리는 말' 타라 .. 이동평균선 정배열 종목 등,https://www.hankyung.com/finance/article/20020...,한국경제,2002.01.28 08:55,8670,\n\t\t\t\t\t\t\t\t종합주가지수가 800선으로 치닫고 있다.\n앨런 그...
1,굿모닝신한증권,[美 '엔론사 분식회계' 파장] '엔론게이트' 후폭풍..폭락하는 국내株價,https://www.hankyung.com/finance/article/20020...,한국경제,2002.01.30 17:34,8670,\n\t\t\t\t\t\t\t\t''엔론 게이트''의 후폭풍이 국내 증시를 강타했다...
2,굿모닝신한증권,"""실적 좋은데..."" 소외株 노려라 .. 대우증권.SK텔레콤 등",https://www.hankyung.com/finance/article/20020...,한국경제,2002.02.20 17:40,8670,\n\t\t\t\t\t\t\t\t국내증시가 예상밖으로 견조한 흐름을 보이고 있다.\...
3,굿모닝신한증권,공모주 청약 한진피앤씨 증권사 배정물량 확정,https://www.hankyung.com/finance/article/20020...,한국경제,2002.03.19 20:12,8670,\n\t\t\t\t\t\t\t\t오는 21-22일 코스닥 등록을 위한 공모주 청약을...
4,굿모닝신한증권,"신한금융, 굿모닝증권 인수 .. 지분 35% 매입 신한증권과 합병키로",https://www.hankyung.com/finance/article/20020...,한국경제,2002.04.04 19:54,8670,"\n\t\t\t\t\t\t\t\t신한금융지주회사가 굿모닝증권을 인수,신한증권과 합병..."
5,굿모닝신한증권,금융권 M&A 바람 .. 증권.은행.보험등 금융주 주도주 부상 가능성,https://www.hankyung.com/finance/article/20020...,한국경제,2002.04.05 15:56,8670,\n\t\t\t\t\t\t\t\t신한금융지주회사의 굿모닝증권 인수방침이 가시화되면서...
6,굿모닝신한증권,[주간 포커스] 진부총리 거취.외환은행장 인선 '촉각',https://www.hankyung.com/economy/article/20020...,한국경제,2002.04.07 17:03,8670,\n\t\t\t\t\t\t\t\t한식(6일)을 전후해 전국에 모처럼 단비가 내렸다....
7,굿모닝신한증권,[종목고르기] 제일제당.LG상사 복수추천,https://www.hankyung.com/finance/article/20020...,한국경제,2002.04.07 18:05,8670,\n\t\t\t\t\t\t\t\t증권사의 이번주 추천종목은 금융 전기전자 자동차 기...
8,굿모닝신한증권,[주식시황] (8일) 외국인 '팔자'에 개인 '사자' 역부족,https://www.hankyung.com/finance/article/20020...,한국경제,2002.04.08 16:30,8670,"\n\t\t\t\t\t\t\t\t종합주가지수가 큰 폭으로 하락,간신히 900선에 턱..."
9,굿모닝신한증권,매수청구가격 주당 6617원 .. '양사 합병비율.절차',https://www.hankyung.com/finance/article/20020...,한국경제,2002.04.08 17:32,8670,\n\t\t\t\t\t\t\t\t신한금융지주회사는 8일 H&Q 등 굿모닝증권 대주주...


In [ ]:
result.to_csv(hk_path + 'hk_news_article_body.csv', encoding = 'utf-8', index = False)